In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('50_Startups.csv')
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, 4].values

# Encoding catagorical data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# from sklearn.preprocessing import LabelEncoder, OneHotEncoder  
# labelencoder_x= LabelEncoder()  
# X[:, 3]= labelencoder_x.fit_transform(X[:, 3])  
# onehotencoder = OneHotEncoder(categorical_features= [3])    
# X = onehotencoder.fit_transform(X).toarray()  

# print(X)

In [3]:
# Avoiding dummy variable trap
X = X[:, 1:]
# Though python libraries will take care of the same

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [5]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression()

In [6]:
Y_pred = regressor.predict(X_test)


In [7]:
# Building the optimal model using backward elimination
# y = b0 + b1 . x1 + b2 . x2 + .... + bn . xn
# The following library does not take care of the b0 constant, we have to add it manually

import statsmodels.api as sm 
X = np.append(arr = np.ones((50, 1)).astype(int), values = X, axis = 1)

In [20]:
# X_opt = X[:, [0, 1, 2, 3, 4, 5]]
# X_opt = np.array(X_opt, dtype=float)
# model = sm.OLS(endog=Y, exog=X_opt)
# result = model.fit()
# result.summary()

def backwardElimination(X, sl):
    numVars = len(X[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(Y, X).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    X = np.delete(X, j, 1)
    regressor_OLS.summary()
        # print('\n')
    return X
 
SL = 0.05
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
X_opt = np.array(X_opt, dtype=float)
X_Modeled = backwardElimination(X_opt, SL)

model = sm.OLS(Y, X_Modeled).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Wed, 04 Oct 2023   Prob (F-statistic):           3.50e-32
Time:                        00:54:37   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.903e+04   2537.897     19.320      0.000    4.39e+04    5.41e+04
x1             0.8543      0.029     29.151      0.000       0.795       0.913
==============================================================================
Omnibus:                       13.727   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.536
Skew:                          -0.911   Prob(JB):                     9.44e-05
Kurtosis:                       5.361   Cond. No.                     1.65e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""